DSE CODING PROJECT


Importing Libraries

In [3]:
import pandas as pd
import geopandas as gpd 
import streamlit as st
import matplotlib.pyplot as plt
import numpy as np

Loading the file

In [4]:
data = pd.read_csv(r"C:\Users\greta\OneDrive\Desktop\GlobalLandTemperaturesByMajorCity.csv")
data.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1849-01-01,26.704,1.435,Abidjan,Côte D'Ivoire,5.63N,3.23W
1,1849-02-01,27.434,1.362,Abidjan,Côte D'Ivoire,5.63N,3.23W
2,1849-03-01,28.101,1.612,Abidjan,Côte D'Ivoire,5.63N,3.23W
3,1849-04-01,26.140,1.387,Abidjan,Côte D'Ivoire,5.63N,3.23W
4,1849-05-01,25.427,1.200,Abidjan,Côte D'Ivoire,5.63N,3.23W


Verifying the cities we need to plot

In [13]:
data.groupby('City').count()

,dt,AverageTemperature,AverageTemperatureUncertainty,Country,Latitude,Longitude
City,,,,,,
Abidjan,1977,1777,1777,1977,1977,1977
Addis Abeba,1965,1679,1679,1965,1965,1965
Ahmadabad,2613,2448,2448,2613,2613,2613
Aleppo,2669,2479,2479,2669,2669,2669
Alexandria,2669,2666,2666,2669,2669,2669
...,...,...,...,...,...,...
Tokyo,2025,2020,2020,2025,2025,2025
Toronto,3239,3141,3141,3239,3239,3239
Umm Durman,1857,1768,1768,1857,1857,1857
